In [1]:
import pandas as pd
import numpy as np
#pd.set_option('display.max_rows', 50000)/

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset=pd.read_csv("/content/drive/MyDrive/ASU/train.csv")
dataset.head(10)



,author,real_label,label,review,product_domain,overall,#words
0,A11OTLEDSW8ZXD,44,44,I have always been a fan of the Lewis and Clar...,books.json.gz,5.0,529
1,A11OTLEDSW8ZXD,44,44,I started reading this book during a three-wee...,books.json.gz,4.0,193
2,A11OTLEDSW8ZXD,44,44,This cable is well-made and feels sturdier tha...,cell_phones_and_accessories.json.gz,4.0,109
3,A11OTLEDSW8ZXD,44,44,"This kibble is expensive, but the cats like it...",pet_supplies.json.gz,5.0,100
4,A11OTLEDSW8ZXD,44,44,This device is exactly what we needed for our ...,electronics.json.gz,4.0,197
5,A11OTLEDSW8ZXD,44,44,These beef hides are quite the treat. I like ...,pet_supplies.json.gz,5.0,130
6,A11OTLEDSW8ZXD,44,44,Reading this memoir was quite a surprise. It ...,books.json.gz,4.0,393
7,A11OTLEDSW8ZXD,44,44,Being a born Hoosier and having graduated from...,books.json.gz,4.0,382
8,A11OTLEDSW8ZXD,44,44,This is a surprisingly good first novel. Whil...,books.json.gz,5.0,140
9,A11OTLEDSW8ZXD,44,44,My first impression of this lens was its weigh...,electronics.json.gz,3.0,348


In [4]:
len(dataset.real_label.unique())

100

In [5]:
dataset_filtered=dataset.loc[:,['real_label','review']]

In [6]:
#Input dataset
dataset_filtered

,real_label,review
0,44,I have always been a fan of the Lewis and Clar...
1,44,I started reading this book during a three-wee...
2,44,This cable is well-made and feels sturdier tha...
3,44,"This kibble is expensive, but the cats like it..."
4,44,This device is exactly what we needed for our ...
...,...,...
49995,46,Dumbo is one of disney's most beloved and clas...
49996,46,I haven't played anygood sport games yet and i...
49997,46,"This game is loads of fun, i really liked this..."
49998,46,Gabriel is one of the funniest comedians i've ...


In [7]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    n_gpu=torch.cuda.device_count()

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [8]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 28.3 MB/s 
     |████████████████████████████████| 7.6 MB 69.4 MB/s 
     |████████████████████████████████| 182 kB 66.5 MB/s 


In [9]:
# from transformers import RobertaTokenizer,BertTokenizer, AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
# from transformers import (
#     AdamW,
#     get_linear_schedule_with_warmup
# )
# """
# # Load the tokenizer.
# print('Loading Roberta tokenizer...')
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)"""

# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# config = AutoConfig.from_pretrained('bert-base-uncased',
#                                             num_labels=101,
#                                             hidden_dropout_prob=0.15)





In [10]:

from transformers import BertTokenizer, AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
from transformers import (
    AdamW,
    get_linear_schedule_with_warmup
)
# from transformers import BertForSequenceClassification, AdamW, BertConfig
# from transformers import get_linear_schedule_with_warmup


output_dir = '/content/drive/MyDrive/ASU/model_save/'

tokenizer = AutoTokenizer.from_pretrained(output_dir+'tokenizer/')
model = AutoModelForSequenceClassification.from_pretrained(output_dir)


# Copy the model to the GPU.
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.15, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [11]:
#Max length

max_len = 0
len_list=[]
# For every sentence...
for r in dataset_filtered.values:
    #print(r)

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(r[1], add_special_tokens=True)
    len_list.append([r[0],len(input_ids)])
    
    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length in training data: ', max_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (646 > 512). Running this sequence through the model will result in indexing errors


Max sentence length in training data:  7283


In [12]:
labels = dataset_filtered.real_label.values
sentences=dataset_filtered.review.values

In [13]:
np.unique(dataset_filtered.real_label.values)


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [14]:
#labels = torch.tensor(labels)

In [15]:
#labels = torch.nn.functional.one_hot(labels, num_classes = 100).to(torch.float)
#print(labels)

In [16]:
# Print the original sentence.
print(' Original: ', sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  I have always been a fan of the Lewis and Clark (L&C) expedition and have read numerous books on that era.  I even road tripped along the Missouri River to the river's source in Montana (bypassing North Dakota and eastern Montana due to repeated tornadoes that summer), stopping along all the historical signs.  I'm a fan because I understand the courage it took to explore unknown lands with potential violent inhabitants, but I also understand the significance of American expansion to the Pacific via the Northwest passage.  Opening up these lands is one of Jefferson's biggest legacies as president.

What Julie Fenster does here is not just summarize the L&C expedition, though.  She describes the young America at the turn of the 19th century.  Pioneers were moving westward, but Spain controlled the western lands. She gives short biographies of the players, the governors, kings and explorers of the era.  It was a time of great hostilities.  Fenster portrays Jefferson as a man f

In [17]:
input_ids = []
attention_masks = []

# For every sentence...
for s in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        s,                      # Sentence to encode.
                        truncation=True,
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        padding='max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        return_overflowing_tokens=False,
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)


In [18]:
input_ids[0]

tensor([  101,  1045,  2031,  2467,  2042,  1037,  5470,  1997,  1996,  4572,
         1998,  5215,  1006,  1048,  1004,  1039,  1007,  5590,  1998,  2031,
         3191,  3365,  2808,  2006,  2008,  3690,  1012,  1045,  2130,  2346,
        21129,  2247,  1996,  5284,  2314,  2000,  1996,  2314,  1005,  1055,
         3120,  1999,  8124,  1006, 11826,  2075,  2167,  7734,  1998,  2789,
         8124,  2349,  2000,  5567, 22668,  2008,  2621,  1007,  1010,  7458,
         2247,  2035,  1996,  3439,  5751,  1012,  1045,  1005,  1049,  1037,
         5470,  2138,  1045,  3305,  1996,  8424,  2009,  2165,  2000,  8849,
         4242,  4915,  2007,  4022,  6355,  4864,  1010,  2021,  1045,  2036,
         3305,  1996,  7784,  1997,  2137,  4935,  2000,  1996,  3534,  3081,
         1996,  4514,  6019,  1012,  3098,  2039,  2122,  4915,  2003,  2028,
         1997,  7625,  1005,  1055,  5221,  4190, 20499,  2004,  2343,  1012,
         2054,  7628, 21713,  6238,  2515,  2182,  2003,  2025, 

In [19]:
attention_masks[0]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [20]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

generator = torch.Generator()
generator.manual_seed(44)

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=generator)

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

45,000 training samples
5,000 validation samples


In [21]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size, # Trains with this batch size.
            num_workers=2
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size, # Evaluate with this batch size.
            num_workers=2
        )

In [22]:
# from transformers import RobertaForSequenceClassification,BertForSequenceClassification, AdamW, BertConfig

# # Load BertForSequenceClassification, the pretrained BERT model with a single 
# # linear classification layer on top. 

# model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

# """
# model = RobertaForSequenceClassification.from_pretrained(
#     "roberta-base", # Use the 12-layer BERT model, with an uncased vocab.
#     num_labels = 100, # The number of output labels--2 for binary classification.
#              # You can increase this for multi-class tasks.   
#     output_attentions = False, # Whether the model returns attentions weights.
#     output_hidden_states = True, # Whether the model returns all hidden-states.
# )"""

# # Tell pytorch to run this model on the GPU.
# model.to(device)

In [23]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [24]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
#total_steps = len(train_dataloader) * epochs
gradient_accumulation_steps=1
n_gpu=1
#total_steps= ((len(train_dataloader) // (batch_size * max(1, n_gpu)))// gradient_accumulation_steps* float(epochs))
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [25]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [26]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [27]:
# x = torch.randn(1)

In [28]:
# x

In [29]:
# r =nn.ReLU()

In [30]:
# r(x)

In [31]:
# torch.cat((x, x, x), 0)

In [32]:
#  eps = 1e-07
#  print(eps)

In [33]:
import torch.nn as nn

In [34]:
import random
seed_val = 44

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 44

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
      #  print(b_input_ids)
      #  print(b_input_mask)
      #  print(b_labels)
        arr_ind=[i for i in range(len(b_labels))]
      #  print(arr_ind)
        counter = 0
        ########
        b_lables_pair_l2 = []
        while(1):
          ind2=np.random.choice(arr_ind)
          ind1=np.random.choice(arr_ind)
          counter =counter + 1
          if(b_labels[ind1].item()!=b_labels[ind2].item() and [ind1,ind2] not in b_lables_pair_l2):
         #    print(ind1,ind2)
             #print(b_labels[ind1].item(),b_labels[ind2].item())
            # arr_ind.remove(ind1)
             arr_ind.remove(ind2)
             b_lables_pair_l2.append([ind1,ind2])
          if(counter>75 or len(arr_ind)<=8):
         #   print(arr_ind)
            break
        #arr_ind = [i for i in range(8)]
        # print(b_labels)
        # print("b_lables_pair_l2",b_lables_pair_l2)
        # print("b_lables_pair_l2",len(b_lables_pair_l2))
        # print("arr_ind",arr_ind)
        # print("arr_ind",len(arr_ind))
        b_input_ids_l1 = torch.cat(tuple([batch[0][ind:ind+1] for ind in arr_ind])).to(device)
        b_input_mask_l1 = torch.cat(tuple([batch[1][ind:ind+1] for ind in arr_ind])).to(device)
        b_labels_l1 = torch.cat(tuple([batch[2][ind:ind+1] for ind in arr_ind])).to(device)
        # print(b_input_ids)
        # print(b_input_ids_l1)

        # print("******")
        # print(batch[0][0:1])
        # print(batch[0][1:2])
        # sum_pair = np.ceil((batch[0][0:1]+batch[0][1:2])/2)
        # avg_pair=sum_pair.type(torch.int64)
        # print(avg_pair)
        b_input_ids_l2=[]
        b_input_mask_l2=[]
        b_labels_l2=[]

        for ind_pair in b_lables_pair_l2:
          i=ind_pair[0]
          j=ind_pair[1]

          sum_pair=np.ceil((batch[0][i:i+1]+batch[0][j:j+1])/2)
          avg_pair=sum_pair.type(torch.int64)
          b_input_ids_l2.append(avg_pair)



          sum_pair_mask=np.ceil((batch[1][i:i+1]+batch[1][j:j+1])/2)
          avg_pair_mask=sum_pair_mask.type(torch.int64)

          b_input_mask_l2.append(avg_pair_mask)

          b_labels_l2.append(torch.tensor([100]))

        b_labels_l2 = torch.cat(tuple(b_labels_l2)).to(device)
        b_input_mask_l2 = torch.cat(tuple(b_input_mask_l2)).to(device)
        b_input_ids_l2 = torch.cat(tuple(b_input_ids_l2)).to(device)
        # print("b_labels_l1",b_labels_l1)
        # print("len(b_labels_l1)",len(b_labels_l1))
        # print("b_labels_l2",b_labels_l2)
        # print("len(b_labels_l2)",len(b_labels_l2))
       

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # In PyTorch, calling `model` will in turn call the model's `forward` 
        # function and pass down the arguments. The `forward` function is 
        # documented here: 
        # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
        # The results are returned in a results object, documented here:
        # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
        # Specifically, we'll get the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation.
        result1 = model(b_input_ids_l1, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask_l1, 
                       labels=b_labels_l1,
                       return_dict=True)
        
        result2 = model(b_input_ids_l2, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask_l2, 
                       labels=b_labels_l2,
                       return_dict=True)
        
 

 
        loss1 = result1.loss
        loss2 = result2.loss
        

        logits1 = result1.logits
        logits2 = result2.logits

        eps = 1e-7
        # t = F.relu()
        # t = torch.log(t +eps)

        kthlogits1 = logits1[: , len(logits1[0])-1:len(logits1[0])].clone()
        kthlogits2 = logits2[: , len(logits2[0])-1:len(logits2[0])].clone()
      #  print("kthlogits before Relu   ",kthlogits1)
        ReLU = nn.ReLU()
        kthlogits1 = ReLU(kthlogits1)
      #  print("kthlogits after Relu   ",kthlogits1)
        kthloss1 = -1 * torch.log(kthlogits1 + eps)
       # print("kthloss  ",kthloss1)
        kthloss1 = torch.sum(kthloss1)
        kthloss1 = torch.sum(kthloss1)/len(b_labels_l1)
       # print("kthloss  ",kthloss1)
        #print("logits : ",logits)
        #print(len(logits))
        kthlogits2 = ReLU(kthlogits2)
        kthloss2 = -1 * torch.log(kthlogits2 + eps)
        kthloss2 = torch.sum(kthloss2)/len(b_labels_l2)
        loss = (2 * loss2) + kthloss2 + loss1 + (0.4 * kthloss1) 

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()
        

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax.
        loss = result.loss
        logits = result.logits
       # print(logits)
        
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
      #  print(label_ids) 
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 2 ========
Training...
  Batch    40  of  2,813.    Elapsed: 0:01:00.
  Batch    80  of  2,813.    Elapsed: 0:02:03.
  Batch   120  of  2,813.    Elapsed: 0:03:05.
  Batch   160  of  2,813.    Elapsed: 0:04:08.
  Batch   200  of  2,813.    Elapsed: 0:05:12.
  Batch   240  of  2,813.    Elapsed: 0:06:14.
  Batch   280  of  2,813.    Elapsed: 0:07:17.
  Batch   320  of  2,813.    Elapsed: 0:08:20.
  Batch   360  of  2,813.    Elapsed: 0:09:23.
  Batch   400  of  2,813.    Elapsed: 0:10:26.
  Batch   440  of  2,813.    Elapsed: 0:11:29.
  Batch   480  of  2,813.    Elapsed: 0:12:31.
  Batch   520  of  2,813.    Elapsed: 0:13:34.
  Batch   560  of  2,813.    Elapsed: 0:14:37.
  Batch   600  of  2,813.    Elapsed: 0:15:40.
  Batch   640  of  2,813.    Elapsed: 0:16:43.
  Batch   680  of  2,813.    Elapsed: 0:17:46.
  Batch   720  of  2,813.    Elapsed: 0:18:49.
  Batch   760  of  2,813.    Elapsed: 0:19:52.
  Batch   800  of  2,813.    Elapsed: 0:20:54.
  Batch   840  of

In [ ]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = '/content/drive/MyDrive/ASU/model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
if not os.path.exists(output_dir+'tokenizer/'):
    os.makedirs(output_dir+'tokenizer/')

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir+'tokenizer/')

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))


Saving model to /content/drive/MyDrive/ASU/model_save/


('/content/drive/MyDrive/ASU/model_save/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/ASU/model_save/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/ASU/model_save/tokenizer/vocab.txt',
 '/content/drive/MyDrive/ASU/model_save/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/ASU/model_save/tokenizer/tokenizer.json')

In [ ]:
# print("Running Validation...")

# t0 = time.time()

# # Put the model in evaluation mode--the dropout layers behave differently
# # during evaluation.
# model.eval()

# # Tracking variables 
# total_eval_accuracy = 0
# total_eval_loss = 0
# nb_eval_steps = 0

# predictions , true_labels = [], []
# count = 1
# # Evaluate data for one epoch
# for batch in validation_dataloader:
    
#     # Unpack this training batch from our dataloader. 
#     #
#     # As we unpack the batch, we'll also copy each tensor to the GPU using 
#     # the `to` method.
#     #
#     # `batch` contains three pytorch tensors:
#     #   [0]: input ids 
#     #   [1]: attention masks
#     #   [2]: labels 
#     b_input_ids = batch[0].to(device)
#     b_input_mask = batch[1].to(device)
#     b_labels = batch[2].to(device)

    
#     # Tell pytorch not to bother with constructing the compute graph during
#     # the forward pass, since this is only needed for backprop (training).
#     with torch.no_grad():        

#         # Forward pass, calculate logit predictions.
#         # token_type_ids is the same as the "segment ids", which 
#         # differentiates sentence 1 and 2 in 2-sentence tasks.
#         result = model(b_input_ids, 
#                         token_type_ids=None, 
#                         attention_mask=b_input_mask,
#                         labels=b_labels,
#                         return_dict=True)

#     # Get the loss and "logits" output by the model. The "logits" are the 
#     # output values prior to applying an activation function like the 
#     # softmax.
#     loss = result.loss
#     logits = result.logits
#     # print(logits)
    
#     # Accumulate the validation loss.
#     total_eval_loss += loss.item()

#     # Move logits and labels to CPU
#     logits = logits.detach().cpu().numpy()
#     logits[:,100] = logits[:,100] + 3
#     label_ids = b_labels.to('cpu').numpy()
#     pred_labels = np.argmax(logits, axis=1)
#   #  print(label_ids) 
#     # Calculate the accuracy for this batch of test sentences, and
#     # accumulate it over all batches.

#     total_eval_accuracy += flat_accuracy(logits, label_ids)

#       # Store predictions and true labels
#     predictions.extend(pred_labels.tolist())
#     true_labels.extend(label_ids.tolist())
#     # if(count == 1):
#     #   break
#     count = count +1

    

# # Report the final accuracy for this validation run.
# avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
# print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

# # Calculate the average loss over all of the batches.
# avg_val_loss = total_eval_loss / len(validation_dataloader)

# # Measure how long the validation run took.
# validation_time = format_time(time.time() - t0)

# print("  Validation Loss: {0:.2f}".format(avg_val_loss))
# print("  Validation took: {:}".format(validation_time))

bias ........................                      percentage

   4   .........................                    .58

In [ ]:
# from sklearn.metrics import classification_report

# result_report= classification_report(true_labels, predictions, digits=3, output_dict=False)
# print(result_report)

In [ ]:
# len(predictions)

In [ ]:
# predictions.count(100)

In [ ]:
# predictions.count(100)/len(predictions)

In [ ]:
# len(true_labels)

In [ ]:
# import numpy as np

# # Function to calculate the accuracy of our predictions vs labels
# def flat_accuracy(preds, labels):
#     pred_flat = np.argmax(preds, axis=1).flatten()
#     labels_flat = labels.flatten()
#     return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# label_ids

In [ ]:
# index = np.argmax(logits, axis=1).flatten()

In [ ]:
# A = np.max(logits, axis=1).flatten()

In [ ]:
# A

In [ ]:
# logits[0]

In [ ]:
# logits[:,100].flatten()

In [ ]:
# logits[:,100]

In [ ]:
# logits[:,100] = logits[:,100] +10

In [ ]:
# logits[0]

In [ ]:
# B = logits[:,100].flatten()+5

In [ ]:
# B

In [ ]:
# B - A

In [ ]:
# logits[0]

In [ ]:
# logits[:,100].flatten()

In [ ]:
# #print(logits)
# for i in logits:
#   print(i[100])

In [ ]:
# import pandas as pd

# # Display floats with two decimal places.
# pd.set_option('precision', 2)

# # Create a DataFrame from our training statistics.
# df_stats = pd.DataFrame(data=training_stats)

# # Use the 'epoch' as the row index.
# df_stats = df_stats.set_index('epoch')

# # A hack to force the column headers to wrap.
# #df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# # Display the table.
# df_stats

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# import seaborn as sns

# # Use plot styling from seaborn.
# sns.set(style='darkgrid')

# # Increase the plot size and font size.
# sns.set(font_scale=1.5)
# plt.rcParams["figure.figsize"] = (12,6)

# # Plot the learning curve.
# plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
# plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# # Label the plot.
# plt.title("Training & Validation Loss")
# plt.xlabel("Epoch")
# plt.ylabel("Loss")
# plt.legend()
# plt.xticks([1, 2, 3, 4])

# plt.show()

In [90]:
import pandas as pd
import regex as re

# Load the dataset into a pandas dataframe.
#df = pd.read_csv("/content/drive/MyDrive/ASU/ood_test_data_small.csv")


#df = pd.read_csv("/content/drive/MyDrive/ASU/stage1_eval_data_ds2.csv")

df = pd.read_csv("/content/drive/MyDrive/ASU/stage1_eval_data_ds3.csv")

In [91]:
# df['flag']=df.loc[:,['real_label','review']].apply(lambda x: re.match("UPDATE*",x['review']),axis=1 )
# #removing updated records

# df=df.loc[:,['real_label','review','flag']].loc[df['flag'].isin([None])]
# df.drop('flag',axis=1,inplace=True)

In [92]:
df = df[['real_label','review']]

In [93]:
df

,real_label,review
0,44,This is the third book in the series with depu...
1,44,This book is nicely divided into two parts: C...
2,44,The character of Edward reminds me of a studen...
3,44,"This may taste ""25% less sugar than the leadin..."
4,44,Having recently taken a level 1 college math c...
...,...,...
99995,175,TVO has penned a novel about an Indian man who...
99996,175,TS has written a novel that begins with the de...
99997,175,AW has writer a novel where Americans high in ...
99998,175,RC has written a SyFy novel about time travel ...


In [94]:
arr_real_labels = [i for i in range(100,10000)]

In [95]:
arr_real_labels

[100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266

In [96]:
df['real_label'] = df['real_label'].replace(arr_real_labels,100)

In [97]:
df=df.loc[df.real_label.values<=100]

In [98]:
df

,real_label,review
0,44,This is the third book in the series with depu...
1,44,This book is nicely divided into two parts: C...
2,44,The character of Edward reminds me of a studen...
3,44,"This may taste ""25% less sugar than the leadin..."
4,44,Having recently taken a level 1 college math c...
...,...,...
99995,100,TVO has penned a novel about an Indian man who...
99996,100,TS has written a novel that begins with the de...
99997,100,AW has writer a novel where Americans high in ...
99998,100,RC has written a SyFy novel about time travel ...


In [99]:
df.groupby(by='real_label').count()

,review
real_label,
0,500
1,500
2,500
3,500
4,500
...,...
96,500
97,500
98,500


In [100]:
labels_list_0 = df['real_label'].tolist()

In [101]:
# for i in range(101):
#   print(i, "     ",c.count(i))

In [102]:

# Create sentence and label lists
#
labels = df.real_label.values
sentence1 = df.review.values
#sentence1=sentences
#labels = dataset_filtered.real_label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent1 in zip(sentence1):
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    #print(sent1)
    encoded_dict = tokenizer.encode_plus(
                        sent1[0],# Sentence to encode.
                        truncation=True,
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        padding='max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        return_overflowing_tokens=False,
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)



# Set the batch size.  
batch_size = 16

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [103]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []
logits_arr=[]
max_logits_arr=[]
logits_total_array = []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions.
      result = model(b_input_ids, 
                     token_type_ids=None, 
                     attention_mask=b_input_mask,
                     return_dict=True)

  logits = result.logits

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  logits_total_array.extend(logits.tolist())
  #logits[:,100] = logits[:,100] + 2.5
  pred_labels = np.argmax(logits, axis=1)
  label_ids = b_labels.to('cpu').numpy()
  max_logits = np.max(logits[:,:100], axis=1)
  # Store predictions and true labels
  predictions.extend(pred_labels.tolist())
  true_labels.extend(label_ids.tolist())
  logits_arr.extend(logits[:,100].tolist())
  max_logits_arr.extend(max_logits.tolist())

print('    DONE.')

Predicting labels for 100,000 test sentences...
    DONE.


In [104]:
d1_predictions = predictions
d1_true_labels = true_labels
d1_logits_arr = logits_arr
d1_max_logits_arr = max_logits_arr

In [105]:
d1_predictions.count(100)

20100

In [106]:
tic = 100000

In [107]:
d1_closed_logits=[]
d1_open_logits=[]
d1_max_closed_logits=[]
d1_max_open_logits=[]
d1_predictions_open=[]
d1_predictions_closed=[]
for i in range(tic):
  if(true_labels[i]==100):
    d1_open_logits.append(d1_logits_arr[i])
    d1_max_open_logits.append(d1_max_logits_arr[i])
    d1_predictions_open.append(d1_predictions[i])
  else:
    d1_closed_logits.append(d1_logits_arr[i])
    d1_max_closed_logits.append(d1_max_logits_arr[i])
    d1_predictions_closed.append(d1_predictions[i])

In [108]:
print(d1_predictions_open.count(100))
print(d1_predictions_closed.count(100))

17704
2396


In [109]:
diff_closed1=[]
for i in range(int(tic/2)):
  diff_closed1.append(5.08 + d1_closed_logits[i] - d1_max_closed_logits[i])
diff_open1=[]
for i in range(int(tic/2)):
  diff_open1.append(5.08 + d1_open_logits[i] - d1_max_open_logits[i])


In [110]:
print(sum(diff_closed1))
print(sorted(diff_closed1)[int(tic/4)])

-24434.730604150318
-1.126081390380861


In [111]:
print(sum(diff_open1))
print(sorted(diff_open1)[int(tic/4)])

177400.2814780572
2.7668247985839827


In [112]:
pos_count = 0 
neg_count = 0
for num in diff_closed1:
    # checking condition
    if num >= 0:
        pos_count += 1
    else:
        neg_count += 1

In [113]:
print(pos_count)
print(neg_count)

6999
43001


In [114]:
pos_count = 0 
neg_count = 0
for num in diff_open1:
    # checking condition
    if num >= 0:
        pos_count += 1
    else:
        neg_count += 1

In [115]:
print(pos_count)
print(neg_count)

34743
15257


In [116]:
logits_total_array_o =logits_total_array

In [117]:
logits_total_array_o = np.array(logits_total_array_o)

In [118]:
logits_total_array_o[:,100] = logits_total_array_o[:,100] + 5.08

In [119]:
predictions_ = np.argmax(logits_total_array_o , axis=1)

In [120]:
np.count_nonzero(predictions_ == 100)

41742

In [121]:
from sklearn.metrics import classification_report

result_report= classification_report(true_labels, predictions_, digits=3, output_dict=False)
print(result_report)

              precision    recall  f1-score   support

           0      0.845     0.700     0.766       500
           1      0.923     0.888     0.905       500
           2      0.671     0.850     0.750       500
           3      0.906     0.962     0.933       500
           4      0.504     0.570     0.535       500
           5      0.632     0.638     0.635       500
           6      0.757     0.842     0.797       500
           7      0.916     0.954     0.934       500
           8      0.919     0.822     0.868       500
           9      0.793     0.722     0.756       500
          10      0.967     0.952     0.960       500
          11      0.852     0.598     0.703       500
          12      0.628     0.974     0.763       500
          13      0.901     0.854     0.877       500
          14      0.771     0.652     0.706       500
          15      0.893     0.956     0.924       500
          16      0.987     0.882     0.931       500
          17      0.940    

In [122]:
#df2 = pd.read_csv("/content/drive/MyDrive/ASU/ood_test_data_small.csv")
df2 = pd.read_csv("/content/drive/MyDrive/ASU/stage1_eval_data_ds3.csv")


In [68]:
df2 = pd.read_csv("/content/drive/MyDrive/ASU/ood_test_data_small.csv")

In [68]:
df2 = pd.read_csv("/content/drive/MyDrive/ASU/stage1_eval_data_ds2.csv")

In [123]:
df2["N_plus_one_prediction"] = predictions_

In [124]:
df2.head(10)

,author,real_label,label,review,product_domain,overall,#words,N_plus_one_prediction
0,A11OTLEDSW8ZXD,44,44,This is the third book in the series with depu...,books.json.gz,4.0,174,100
1,A11OTLEDSW8ZXD,44,44,This book is nicely divided into two parts: C...,books.json.gz,4.0,509,100
2,A11OTLEDSW8ZXD,44,44,The character of Edward reminds me of a studen...,books.json.gz,4.0,312,44
3,A11OTLEDSW8ZXD,44,44,"This may taste ""25% less sugar than the leadin...",grocery_and_gourmet_food.json.gz,4.0,85,100
4,A11OTLEDSW8ZXD,44,44,Having recently taken a level 1 college math c...,books.json.gz,5.0,131,44
5,A11OTLEDSW8ZXD,44,44,I am a Blogger fan myself and read this book t...,books.json.gz,4.0,254,100
6,A11OTLEDSW8ZXD,44,44,The goal of this video is to prove or disprove...,movies_and_tv.json.gz,4.0,392,44
7,A11OTLEDSW8ZXD,44,44,This leash is made to last. It's made from on...,pet_supplies.json.gz,5.0,267,44
8,A11OTLEDSW8ZXD,44,44,It's the story of Bandit the cat moving with h...,books.json.gz,4.0,149,100
9,A11OTLEDSW8ZXD,44,44,I bought this canned food when Amazon recently...,pet_supplies.json.gz,5.0,158,81


In [125]:
df2['N_plus_one_prediction'] = df2['N_plus_one_prediction'].replace([100], 'OOD')

In [126]:
df2.head(10)

,author,real_label,label,review,product_domain,overall,#words,N_plus_one_prediction
0,A11OTLEDSW8ZXD,44,44,This is the third book in the series with depu...,books.json.gz,4.0,174,OOD
1,A11OTLEDSW8ZXD,44,44,This book is nicely divided into two parts: C...,books.json.gz,4.0,509,OOD
2,A11OTLEDSW8ZXD,44,44,The character of Edward reminds me of a studen...,books.json.gz,4.0,312,44
3,A11OTLEDSW8ZXD,44,44,"This may taste ""25% less sugar than the leadin...",grocery_and_gourmet_food.json.gz,4.0,85,OOD
4,A11OTLEDSW8ZXD,44,44,Having recently taken a level 1 college math c...,books.json.gz,5.0,131,44
5,A11OTLEDSW8ZXD,44,44,I am a Blogger fan myself and read this book t...,books.json.gz,4.0,254,OOD
6,A11OTLEDSW8ZXD,44,44,The goal of this video is to prove or disprove...,movies_and_tv.json.gz,4.0,392,44
7,A11OTLEDSW8ZXD,44,44,This leash is made to last. It's made from on...,pet_supplies.json.gz,5.0,267,44
8,A11OTLEDSW8ZXD,44,44,It's the story of Bandit the cat moving with h...,books.json.gz,4.0,149,OOD
9,A11OTLEDSW8ZXD,44,44,I bought this canned food when Amazon recently...,pet_supplies.json.gz,5.0,158,81


In [127]:
combined_df = df2
number_of_known_labels = 100

In [128]:

def compute_precision_recall_for_known_classes(given_label):
    TP = combined_df[(combined_df["real_label"] == given_label) & (combined_df["N_plus_one_prediction"] == given_label)].shape[0]
    FP = combined_df[(combined_df["real_label"] != given_label) & (combined_df["N_plus_one_prediction"] == given_label)].shape[0]
    FN = combined_df[(combined_df["real_label"] == given_label) & (combined_df["N_plus_one_prediction"] != given_label)].shape[0]
    if(TP == 0):
        P = 0
        R = 0
    else:
        P = round(100*TP/(TP + FP), 2)
        R = round(100*TP/(TP + FN),2)
    return P, R

In [129]:
def compute_precision_recall_for_OOD():
    TP = combined_df[(combined_df["real_label"] >= number_of_known_labels) & (combined_df["N_plus_one_prediction"] == "OOD")].shape[0]
    FP = combined_df[(combined_df["real_label"] < number_of_known_labels) & (combined_df["N_plus_one_prediction"] == "OOD")].shape[0]
    FN = combined_df[(combined_df["real_label"] >= number_of_known_labels) & (combined_df["N_plus_one_prediction"] != "OOD")].shape[0]
    if(TP == 0):
        P = 0
        R = 0
    else:
        P = 100*TP/(TP + FP)
        R = 100*TP/(TP + FN)
    return P, R

In [130]:




def compute_performance_metrics():
    from statistics import mean
    
    precisions = []
    recalls = []
    for known_label in range(number_of_known_labels):
        p, r = compute_precision_recall_for_known_classes(known_label)
        precisions.append(p)
        recalls.append(r)
    
    precision_known, recall_known = round(mean(precisions),2), round(mean(recalls),2)
    f1_known = round(2*precision_known*recall_known/(precision_known + recall_known),2)
    
    precision_ood, recall_ood = compute_precision_recall_for_OOD()
    f1_ood = round(2*precision_ood*recall_ood/(precision_ood + recall_ood),2)
    
    precisions.append(precision_ood)
    recalls.append(recall_ood)
    
    precision, recall = round(mean(precisions),2), round(mean(recalls),2)
    f1 = round(2*precision*recall/(precision + recall),2)
    
    return {
        "known": {
            "precision": precision_known,
            "recall": recall_known,
            "f1": f1_known,
        },
        "ood": {
            "precision": round(precision_ood,2),
            "recall": round(recall_ood,2),
            "f1": f1_ood,
        },
        "overall": {
            "precision": precision,
            "recall": recall,
            "f1": f1,
        }
        
    }

In [131]:
def print_performance_metrics():
    performance_dict = compute_performance_metrics()
    print(performance_dict["known"]["precision"],performance_dict["known"]["recall"],performance_dict["known"]["f1"], end = " ")
    print(performance_dict["ood"]["precision"],performance_dict["ood"]["recall"],performance_dict["ood"]["f1"], end = " ")
    print(performance_dict["overall"]["precision"],performance_dict["overall"]["recall"],performance_dict["overall"]["f1"], end = " ")
    return performance_dict["overall"]["f1"]

In [132]:
print_performance_metrics()

75.42 83.78 79.38 83.23 69.49 75.74 75.49 83.64 79.36 

79.36

In [133]:
list_of_f1=[]
for i in range(100,401):
  logits_total_array_o =logits_total_array
  logits_total_array_o = np.array(logits_total_array_o)
  logits_total_array_o[:,100] = logits_total_array_o[:,100] + (i/100)
  predictions_ = np.argmax(logits_total_array_o , axis=1)
  df2["N_plus_one_prediction"] = predictions_
  df2['N_plus_one_prediction'] = df2['N_plus_one_prediction'].replace([100], 'OOD')
  print("\n",i/100,"    ",end="")
  list_of_f1.append(print_performance_metrics())
 # print_performance_metrics()



 1.0     63.88 89.08 74.4 87.47 41.53 56.32 64.11 88.61 74.39 
 1.01     63.89 89.07 74.41 87.46 41.59 56.37 64.13 88.6 74.4 
 1.02     63.91 89.07 74.42 87.45 41.64 56.42 64.14 88.6 74.41 
 1.03     63.93 89.06 74.43 87.44 41.69 56.46 64.16 88.59 74.42 
 1.04     63.96 89.05 74.45 87.43 41.78 56.54 64.19 88.58 74.44 
 1.05     63.98 89.05 74.46 87.4 41.83 56.58 64.21 88.58 74.45 
 1.06     64.0 89.03 74.47 87.37 41.88 56.62 64.23 88.57 74.46 
 1.07     64.02 89.03 74.48 87.37 41.94 56.67 64.25 88.56 74.47 
 1.08     64.05 89.02 74.5 87.37 42.0 56.73 64.28 88.56 74.49 
 1.09     64.08 89.01 74.52 87.37 42.08 56.81 64.31 88.55 74.51 
 1.1     64.1 89.01 74.53 87.36 42.14 56.85 64.33 88.55 74.52 
 1.11     64.14 89.01 74.56 87.36 42.24 56.94 64.37 88.54 74.54 
 1.12     64.17 89.0 74.57 87.35 42.3 57.0 64.4 88.54 74.56 
 1.13     64.19 88.99 74.58 87.34 42.36 57.05 64.42 88.53 74.57 
 1.14     64.21 88.99 74.6 87.33 42.42 57.1 64.44 88.52 74.58 
 1.15     64.24 88.98 74.61 87.33 42.49 5

In [134]:
max(list_of_f1)

78.18

In [135]:
list_of_f1=[]
for i in range(400,501):
  logits_total_array_o =logits_total_array
  logits_total_array_o = np.array(logits_total_array_o)
  logits_total_array_o[:,100] = logits_total_array_o[:,100] + (i/100)
  predictions_ = np.argmax(logits_total_array_o , axis=1)
  df2["N_plus_one_prediction"] = predictions_
  df2['N_plus_one_prediction'] = df2['N_plus_one_prediction'].replace([100], 'OOD')
  print("\n",i/100,"    ",end="")
  list_of_f1.append(print_performance_metrics())
 # print_performance_metrics()


 4.0     71.48 86.31 78.2 85.19 61.04 71.12 71.62 86.06 78.18 
 4.01     71.51 86.3 78.21 85.2 61.11 71.17 71.65 86.05 78.19 
 4.02     71.55 86.28 78.23 85.19 61.2 71.23 71.69 86.03 78.21 
 4.03     71.57 86.25 78.23 85.15 61.24 71.24 71.7 86.01 78.21 
 4.04     71.61 86.24 78.25 85.16 61.33 71.3 71.74 86.0 78.23 
 4.05     71.64 86.23 78.26 85.15 61.41 71.35 71.78 85.98 78.24 
 4.06     71.68 86.22 78.28 85.14 61.48 71.4 71.81 85.97 78.25 
 4.07     71.7 86.2 78.28 85.12 61.53 71.43 71.83 85.95 78.26 
 4.08     71.73 86.18 78.29 85.11 61.61 71.48 71.86 85.94 78.27 
 4.09     71.77 86.15 78.31 85.09 61.69 71.52 71.9 85.91 78.28 
 4.1     71.79 86.14 78.31 85.08 61.74 71.55 71.92 85.9 78.29 
 4.11     71.81 86.12 78.32 85.06 61.79 71.58 71.94 85.88 78.29 
 4.12     71.86 86.11 78.34 85.06 61.89 71.65 71.99 85.87 78.32 
 4.13     71.9 86.1 78.36 85.05 61.96 71.69 72.03 85.86 78.34 
 4.14     71.93 86.08 78.37 85.04 62.03 71.74 72.06 85.84 78.35 
 4.15     71.95 86.05 78.37 85.01 62.1 7

In [136]:
max(list_of_f1)

79.32

In [137]:
print("done")

done


In [138]:
list_of_f1=[]
for i in range(500,701):
  logits_total_array_o =logits_total_array
  logits_total_array_o = np.array(logits_total_array_o)
  logits_total_array_o[:,100] = logits_total_array_o[:,100] + (i/100)
  predictions_ = np.argmax(logits_total_array_o , axis=1)
  df2["N_plus_one_prediction"] = predictions_
  df2['N_plus_one_prediction'] = df2['N_plus_one_prediction'].replace([100], 'OOD')
  print("\n",i/100,"    ",end="")
  list_of_f1.append(print_performance_metrics())
 # print_performance_metrics()


 5.0     75.09 84.1 79.34 83.49 68.79 75.43 75.18 83.95 79.32 
 5.01     75.13 84.07 79.35 83.46 68.85 75.46 75.21 83.92 79.33 
 5.02     75.16 84.02 79.34 83.42 68.94 75.49 75.25 83.87 79.33 
 5.03     75.21 83.98 79.35 83.4 69.04 75.55 75.29 83.84 79.34 
 5.04     75.26 83.93 79.36 83.35 69.13 75.58 75.34 83.79 79.34 
 5.05     75.3 83.9 79.37 83.32 69.22 75.62 75.38 83.75 79.34 
 5.06     75.34 83.85 79.37 83.29 69.31 75.66 75.42 83.71 79.35 
 5.07     75.38 83.82 79.38 83.26 69.4 75.7 75.46 83.68 79.36 
 5.08     75.42 83.78 79.38 83.23 69.49 75.74 75.49 83.64 79.36 
 5.09     75.46 83.74 79.38 83.2 69.57 75.78 75.54 83.6 79.37 
 5.1     75.49 83.7 79.38 83.16 69.64 75.8 75.57 83.56 79.36 
 5.11     75.53 83.64 79.38 83.11 69.72 75.83 75.61 83.51 79.36 
 5.12     75.58 83.6 79.39 83.07 69.82 75.87 75.65 83.46 79.36 
 5.13     75.62 83.56 79.39 83.04 69.91 75.91 75.7 83.42 79.37 
 5.14     75.67 83.51 79.4 83.01 70.0 75.95 75.74 83.37 79.37 
 5.15     75.71 83.47 79.4 82.97 70.08 7

In [139]:
max(list_of_f1)

79.4

In [140]:
print("done epochs 7 8 varun 3")

done epochs 7 8 varun 3


In [141]:
logits_total_array_o = logits_total_array_o.tolist()

In [142]:
df2['logits'] = logits_total_array_o

In [143]:
df2.to_csv("/content/sample_data/logits_3_varun.csv")

In [ ]:
predictions_ = np.argmax(logits_total_array , axis=1)

In [ ]:
len(logits_total_array)

20000

In [ ]:
from sklearn.metrics import classification_report

result_report= classification_report(true_labels, predictions, digits=3, output_dict=False)
print(result_report)

              precision    recall  f1-score   support

           0      0.530     0.790     0.635       100
           1      0.959     0.930     0.944       100
           2      0.802     0.850     0.825       100
           3      0.960     0.970     0.965       100
           4      0.676     0.460     0.548       100
           5      0.824     0.700     0.757       100
           6      0.889     0.400     0.552       100
           7      0.790     0.980     0.875       100
           8      0.803     0.940     0.866       100
           9      0.631     0.820     0.713       100
          10      0.960     0.950     0.955       100
          11      0.690     0.800     0.741       100
          12      0.990     0.970     0.980       100
          13      0.954     0.830     0.888       100
          14      0.519     0.670     0.585       100
          15      0.913     0.950     0.931       100
          16      0.895     0.940     0.917       100
          17      0.843    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
d_predictions = predictions
d_true_labels = true_labels
d_logits_arr = logits_arr
d_max_logits_arr = max_logits_arr

In [ ]:
d_logits_arr
d_threshold_arr=[]
for i in range(len(d_logits_arr)):
  if(d_logits_arr[i]>1):
    d_threshold_arr.append(d_true_labels[i])

In [ ]:
len(d_threshold_arr)

1499

In [ ]:
d_threshold_arr.count(100)

471

In [ ]:
np.unique(d_predictions)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100])

In [ ]:
np.unique(d_true_labels)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100])

In [ ]:
len(d_logits_arr)

20000

In [ ]:
len(d_predictions)

20000

In [ ]:
d_predictions.count(100)

9327

In [ ]:
d_true_labels == labels_list_0

True

In [ ]:
d_predictions.count(100)/10000

0.9327

In [ ]:
#all classes
from sklearn.metrics import classification_report

result_report= classification_report(true_labels, predictions, digits=3, output_dict=False)
print(result_report)

              precision    recall  f1-score   support

           0      0.607     0.680     0.642       100
           1      0.821     0.870     0.845       100
           2      0.532     0.830     0.648       100
           3      0.904     0.940     0.922       100
           4      0.221     0.300     0.254       100
           5      0.722     0.570     0.637       100
           6      0.611     0.330     0.429       100
           7      0.853     0.930     0.890       100
           8      0.729     0.940     0.821       100
           9      0.663     0.690     0.676       100
          10      0.887     0.940     0.913       100
          11      0.615     0.750     0.676       100
          12      0.687     0.920     0.786       100
          13      0.830     0.780     0.804       100
          14      0.519     0.560     0.538       100
          15      0.908     0.790     0.845       100
          16      0.895     0.850     0.872       100
          17      0.868    

In [ ]:
#all classes
from sklearn.metrics import classification_report

result_report= classification_report(true_labels, predictions, digits=3, output_dict=False)
print(result_report)

              precision    recall  f1-score   support

           0      0.507     0.730     0.598       100
           1      0.744     0.930     0.827       100
           2      0.483     0.830     0.610       100
           3      0.839     0.940     0.887       100
           4      0.193     0.340     0.246       100
           5      0.637     0.580     0.607       100
           6      0.576     0.340     0.428       100
           7      0.785     0.950     0.860       100
           8      0.655     0.950     0.776       100
           9      0.592     0.740     0.658       100
          10      0.895     0.940     0.917       100
          11      0.588     0.770     0.667       100
          12      0.625     0.950     0.754       100
          13      0.767     0.890     0.824       100
          14      0.522     0.590     0.554       100
          15      0.890     0.890     0.890       100
          16      0.923     0.840     0.880       100
          17      0.779    

In [ ]:
d_closed_logits=[]
d_open_logits=[]
d_max_closed_logits=[]
d_max_open_logits=[]
d_predictions_open=[]
d_predictions_closed=[]
for i in range(20000):
  if(true_labels[i]==100):
    d_open_logits.append(logits_arr[i])
    d_max_open_logits.append(d_max_logits_arr[i])
    d_predictions_open.append(d_predictions[i])
  else:
    d_closed_logits.append(logits_arr[i])
    d_max_closed_logits.append(d_max_logits_arr[i])
    d_predictions_closed.append(d_predictions[i])

In [ ]:
print(d_predictions_closed[800])
print(d_max_closed_logits[800])
print(d_closed_logits[800])

100
10.72829532623291
10.966748237609863


In [ ]:
d_predictions_open.count(100)

5865

In [ ]:
true_labels[0]

44

In [ ]:
sum(d_closed_logits)/10000

-0.4986603997487575

In [ ]:
sum(d_open_logits)/10000

-0.6508012606155127

In [ ]:
sum(d_max_closed_logits)/10000

9.227002764344215

In [ ]:
sum(d_max_open_logits)/10000

7.152337114357948

In [ ]:
d_open_logits[0]

In [ ]:
d_max_open_logits[0]

In [ ]:
diff_open=[]
for i in range(10000):
  diff_open.append(d_max_open_logits[i]-d_open_logits[i])
diff_closed=[]
for i in range(10000):
  diff_closed.append(d_max_closed_logits[i]-d_closed_logits[i])

In [ ]:
sum(diff_open)

78031.38374973461

In [ ]:
sum(diff_closed)

97256.63164092973

In [ ]:
closed_logits.sort()


In [ ]:
len(closed_logits)

In [ ]:
closed_logits[7500]

In [ ]:
open_logits.sort()

In [ ]:
open_logits[7500]

In [ ]:
df2 = pd.read_csv("/content/drive/MyDrive/ASU/ood_test_data_small.csv")

In [ ]:
df2["N_plus_one_prediction"] = predictions

In [ ]:
df2.head(10)

,author,real_label,label,review,product_domain,overall,#words,N_plus_one_prediction
0,A11OTLEDSW8ZXD,44,44,"I bought this battery charger a year ago, when...",automotive.json.gz,5.0,330,100
1,A11OTLEDSW8ZXD,44,44,I bought this because with four geriatric vehi...,automotive.json.gz,5.0,170,44
2,A11OTLEDSW8ZXD,44,44,I was a tad skeptical of the outcome after see...,automotive.json.gz,4.0,315,44
3,A11OTLEDSW8ZXD,44,44,I insist on every one of my vehicles to have a...,automotive.json.gz,5.0,167,44
4,A11OTLEDSW8ZXD,44,44,This is a very well-made top grain cow leather...,automotive.json.gz,5.0,191,44
5,A11OTLEDSW8ZXD,44,44,At first I was pleased to get this roller shad...,automotive.json.gz,4.0,137,90
6,A11OTLEDSW8ZXD,44,44,"This strap appears well-made, with inch-thick ...",automotive.json.gz,4.0,84,44
7,A11OTLEDSW8ZXD,44,44,"For a quick, overall shine on chrome and plast...",automotive.json.gz,4.0,110,44
8,A11OTLEDSW8ZXD,44,44,"This balaclava comes in a small, bubble-wrap p...",automotive.json.gz,5.0,161,44
9,A11OTLEDSW8ZXD,44,44,This is an impressive car jump battery pack wi...,automotive.json.gz,5.0,317,44


In [ ]:
df2['N_plus_one_prediction'] = df2['N_plus_one_prediction'].replace([100], 'OOD')

In [ ]:
df2.head(10)

,author,real_label,label,review,product_domain,overall,#words,N_plus_one_prediction
0,A11OTLEDSW8ZXD,44,44,"I bought this battery charger a year ago, when...",automotive.json.gz,5.0,330,OOD
1,A11OTLEDSW8ZXD,44,44,I bought this because with four geriatric vehi...,automotive.json.gz,5.0,170,44
2,A11OTLEDSW8ZXD,44,44,I was a tad skeptical of the outcome after see...,automotive.json.gz,4.0,315,44
3,A11OTLEDSW8ZXD,44,44,I insist on every one of my vehicles to have a...,automotive.json.gz,5.0,167,44
4,A11OTLEDSW8ZXD,44,44,This is a very well-made top grain cow leather...,automotive.json.gz,5.0,191,44
5,A11OTLEDSW8ZXD,44,44,At first I was pleased to get this roller shad...,automotive.json.gz,4.0,137,90
6,A11OTLEDSW8ZXD,44,44,"This strap appears well-made, with inch-thick ...",automotive.json.gz,4.0,84,44
7,A11OTLEDSW8ZXD,44,44,"For a quick, overall shine on chrome and plast...",automotive.json.gz,4.0,110,44
8,A11OTLEDSW8ZXD,44,44,"This balaclava comes in a small, bubble-wrap p...",automotive.json.gz,5.0,161,44
9,A11OTLEDSW8ZXD,44,44,This is an impressive car jump battery pack wi...,automotive.json.gz,5.0,317,44


In [ ]:
combined_df = df2
number_of_known_labels = 100

In [ ]:

def compute_precision_recall_for_known_classes(given_label):
    TP = combined_df[(combined_df["real_label"] == given_label) & (combined_df["N_plus_one_prediction"] == given_label)].shape[0]
    FP = combined_df[(combined_df["real_label"] != given_label) & (combined_df["N_plus_one_prediction"] == given_label)].shape[0]
    FN = combined_df[(combined_df["real_label"] == given_label) & (combined_df["N_plus_one_prediction"] != given_label)].shape[0]
    if(TP == 0):
        P = 0
        R = 0
    else:
        P = round(100*TP/(TP + FP), 2)
        R = round(100*TP/(TP + FN),2)
    return P, R

In [ ]:
def compute_precision_recall_for_OOD():
    TP = combined_df[(combined_df["real_label"] >= number_of_known_labels) & (combined_df["N_plus_one_prediction"] == "OOD")].shape[0]
    FP = combined_df[(combined_df["real_label"] < number_of_known_labels) & (combined_df["N_plus_one_prediction"] == "OOD")].shape[0]
    FN = combined_df[(combined_df["real_label"] >= number_of_known_labels) & (combined_df["N_plus_one_prediction"] != "OOD")].shape[0]
    if(TP == 0):
        P = 0
        R = 0
    else:
        P = 100*TP/(TP + FP)
        R = 100*TP/(TP + FN)
    return P, R

In [ ]:




def compute_performance_metrics():
    from statistics import mean
    
    precisions = []
    recalls = []
    for known_label in range(number_of_known_labels):
        p, r = compute_precision_recall_for_known_classes(known_label)
        precisions.append(p)
        recalls.append(r)
    
    precision_known, recall_known = round(mean(precisions),2), round(mean(recalls),2)
    f1_known = round(2*precision_known*recall_known/(precision_known + recall_known),2)
    
    precision_ood, recall_ood = compute_precision_recall_for_OOD()
    f1_ood = round(2*precision_ood*recall_ood/(precision_ood + recall_ood),2)
    
    precisions.append(precision_ood)
    recalls.append(recall_ood)
    
    precision, recall = round(mean(precisions),2), round(mean(recalls),2)
    f1 = round(2*precision*recall/(precision + recall),2)
    
    return {
        "known": {
            "precision": precision_known,
            "recall": recall_known,
            "f1": f1_known,
        },
        "ood": {
            "precision": round(precision_ood,2),
            "recall": round(recall_ood,2),
            "f1": f1_ood,
        },
        "overall": {
            "precision": precision,
            "recall": recall,
            "f1": f1,
        }
        
    }

In [ ]:
def print_performance_metrics():
    performance_dict = compute_performance_metrics()
    print(performance_dict["known"]["precision"],performance_dict["known"]["recall"],performance_dict["known"]["f1"], end = " ")
    print(performance_dict["ood"]["precision"],performance_dict["ood"]["recall"],performance_dict["ood"]["f1"], end = " ")
    print(performance_dict["overall"]["precision"],performance_dict["overall"]["recall"],performance_dict["overall"]["f1"], end = " ")

In [ ]:
print_performance_metrics()

69.45 70.23 69.84 74.12 69.13 71.54 69.49 70.22 69.85 

In [ ]:
   macro avg      0.517     0.797     0.607     20000
weighted avg      0.684     0.506     0.473     20000

In [ ]:
np.unique(predictions)

In [ ]:
np.unique(true_labels)

In [ ]:
len(predictions)

In [ ]:
predictions.count(100)

In [ ]:
predictions.count(100)/len(predictions)

In [ ]:
#known classes
from sklearn.metrics import classification_report

result_report= classification_report(true_labels, predictions, digits=3, output_dict=False)
print(result_report)

In [ ]:
#unknown classes
from sklearn.metrics import classification_report

result_report= classification_report(true_labels, predictions, digits=3, output_dict=False)
print(result_report)

In [ ]:
import json
your_json_string = result_report
json_object = json.dumps(your_json_string, indent=4, ensure_ascii=False)

with open("results_1222715130.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:

predictions_ , true_labels_ = [], []
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions.
      result = model(b_input_ids, 
                     token_type_ids=None, 
                     attention_mask=b_input_mask,
                     return_dict=True)

  logits = result.logits

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  print(logits[1])
  pred_labels = np.argmax(logits, axis=1)
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions_.extend(pred_labels.tolist())
  true_labels_.extend(label_ids.tolist())
  print(predictions_)
  print(true_labels_)
  break

In [ ]:
from sklearn.metrics import classification_report

result_report= classification_report(true_labels, predictions, digits=3)
print(result_report)

In [ ]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = '/content/drive/MyDrive/ASU/model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))


In [ ]:
!ls -l --block-size=K /content/drive/MyDrive/ASU/model_save/

In [ ]:
!ls -l --block-size=M /content/drive/MyDrive/ASU/model_save/pytorch_model.bin

In [ ]:
!pip install transformers
from transformers import BertTokenizer, AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
from transformers import (
    AdamW,
    get_linear_schedule_with_warmup
)
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

In [ ]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
# Load a trained model and vocabulary that you have fine-tuned
#model = RobertaForSequenceClassification.from_pretrained(output_dir)
#tokenizer = RobertaForSequenceClassification.from_pretrained(output_dir)


output_dir = '/content/drive/MyDrive/ASU/model_save/'

tokenizer = AutoTokenizer.from_pretrained(output_dir+'tokenizer/')
model = AutoModelForSequenceClassification.from_pretrained(output_dir)


# Copy the model to the GPU.
model.to(device)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')